In [1]:
import os
import pandas as pd
import numpy as np
from nltk.tokenize import TweetTokenizer
from collections import Counter

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Bidirectional, Dropout, Activation, Conv1D, GlobalMaxPooling1D
from keras import regularizers, initializers



Using TensorFlow backend.


In [5]:
def create_dictionary(texts, vocab_size):
    """
    Creates a dictionary that maps words to ids. More frequent words have lower ids.
    The dictionary contains at the vocab_size-1 most frequent words (and a placeholder '<unk>' for unknown words).
    The place holder has the id 0.
    """
    counter = Counter()
    for tokens in texts:
        counter.update(tokens)
    vocab = [w for w, c in counter.most_common(vocab_size - 1)]
    word_to_id = {w: (i + 1) for i, w in enumerate(vocab)}
    word_to_id[UNKNOWN_TOKEN] = 0
    return word_to_id


def to_ids(words, dictionary):
    """
    Takes a list of words and converts them to ids using the word2id dictionary.
    """
    ids = []
    for word in words:
        ids.append(dictionary.get(word, dictionary[UNKNOWN_TOKEN]))
    return ids


def read_data(train_file, dev_file):
    tokenizer = TweetTokenizer()
    trainDF = pd.read_csv(train_file, sep='\t')
    devDF = pd.read_csv(dev_file, sep='\t')

    allDF = pd.concat([trainDF, devDF], ignore_index=True)
    allDF = allDF.reindex(np.random.permutation(allDF.index))
    allDF.insert(1, 'tweet_tokenized', (allDF['Tweet'].apply(lambda x: tokenizer.tokenize(x))))

    word2id = create_dictionary(allDF["tweet_tokenized"], VOCAB_SIZE)

    allDF.insert(1, 'tweet_ids', (allDF['Tweet'].apply(lambda x: to_ids(x, dictionary=word2id))))

    allDF['all'] = allDF.iloc[:, -11:].values.tolist()

    # calc class weights
    # class_weights = compute_class_weights(allDF)
    class_weights = None    
    # print(class_weights)

    total = len(allDF)
    trainend = int(total * 0.8)
    devend = trainend + int(total * 0.1)
    return allDF.iloc[:trainend, :], allDF.iloc[trainend:devend, :], allDF.iloc[devend:, :], class_weights


def evaluate(predictions, y_test):
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    all_correct = 0
    for i, pred in enumerate(predictions):
        for j, em in enumerate(pred):
            if em >= 0.5:
                if y_test[i][j] == 1:
                    tp += 1
                else:
                    fp += 1
            if em <= 0.5:
                if y_test[i][j] == 1:
                    fn += 1
                else:
                    tn += 1
            if tp + tn == 11:
                all_correct += 1
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * (precision * recall) / (precision + recall)

    print("F1: {}\nPrecision: {}\nRecall: {}\nCompletely correct: {}".format(f1, precision, recall, all_correct))


In [6]:
data_dir = 'D:/3_Programming/1_Studium/Python/SemEval2018_Task1_5/data/'
train_file = os.path.join(data_dir, '2018-E-c-En-train.txt')
dev_file = os.path.join(data_dir, '2018-E-c-En-dev.txt')

VOCAB_SIZE = 100000
MAX_LEN = 100
BATCH_SIZE = 32
EMBEDDING_SIZE = 100
HIDDEN_SIZE = 50
EPOCHS = 10  # Standard 10
UNKNOWN_TOKEN = "<unk>"
EMOTIONS = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love',
            'optimism', 'pessimism', 'sadness', 'surprise', 'trust']

In [56]:
trainDF, devDF, testDF, class_weights = read_data(train_file, dev_file)
x_train = sequence.pad_sequences(np.array(trainDF['tweet_ids']), maxlen=MAX_LEN)
x_dev = sequence.pad_sequences(np.array(devDF['tweet_ids']), maxlen=MAX_LEN)
x_test = sequence.pad_sequences(np.array(testDF['tweet_ids']), maxlen=MAX_LEN)
y_train = np.array([trainDF['all']])[0]
y_dev = np.array([devDF['all']])[0]
y_test = np.array([testDF['all']])[0]


In [57]:
cnn_model = Sequential()
cnn_model.add(Embedding(VOCAB_SIZE, EMBEDDING_SIZE))
cnn_model.add(Conv1D(2 * HIDDEN_SIZE,
                     kernel_size=3,
                     activation='tanh',
                     strides=1,
                     padding='valid',
                     ))
cnn_model.add(GlobalMaxPooling1D())
cnn_model.add(Dropout(0.5))
cnn_model.add(Dense(11, activation='sigmoid')) #11 = no of classes

In [67]:
from keras.optimizers import SGD, adam

opt = adam(lr=0.01)
cnn_model.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'],
                  # sample_weight_mode='temporal'
                  )


In [68]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

early_stopper = EarlyStopping(monitor='val_acc', patience=5, mode='max')
checkpoint = ModelCheckpoint(data_dir+'model.m', save_best_only=True, monitor='val_acc', mode='max')


In [69]:
def compute_class_weights(y_train):
    class_weights = np.sum(y_train, axis=0)
    sum_class_weights = np.sum(y_train)
    
    class_weights = [((sum_class_weights - i) / sum_class_weights)**10 for i in class_weights]
    
    return class_weights
compute_class_weights(y_train)

In [70]:
def compute_sample_weights(y_train, factor):
    print(y_train)
    class_weights = np.array(y_train)
    class_weights[class_weights == 1] = factor
    class_weights[class_weights == 0] = 1
    return np.reshape(class_weights, class_weights.shape + (1,))

# sample_weights = compute_sample_weights(y_train, 2)
# y_train = np.reshape(y_train, y_train.shape + (1,))

In [71]:
cnn_model.fit(
    x_train,
    y_train,
    batch_size=BATCH_SIZE,
    callbacks=[early_stopper, checkpoint],
    epochs=EPOCHS,
    validation_data=(x_dev, y_dev),
    class_weight=class_weights,
    # sample_weight=sample_weights,
    verbose=1
)


Train on 6179 samples, validate on 772 samples
Epoch 1/10


  32/6179 [..............................] - ETA: 7:00 - loss: 0.6989 - acc: 0.4773

  96/6179 [..............................] - ETA: 2:24 - loss: 0.6600 - acc: 0.6411

 160/6179 [..............................] - ETA: 1:29 - loss: 0.6016 - acc: 0.7017

 224/6179 [>.............................] - ETA: 1:05 - loss: 0.5772 - acc: 0.7208

...

5952/6179 [===========================>..] - ETA: 0s - loss: 0.4304 - acc: 0.8098

6016/6179 [============================>.] - ETA: 0s - loss: 0.4304 - acc: 0.8100

6080/6179 [============================>.] - ETA: 0s - loss: 0.4313 - acc: 0.8095

6144/6179 [============================>.] - ETA: 0s - loss: 0.4313 - acc: 0.8096

6179/6179 [==============================] - 8s 1ms/step - loss: 0.4310 - acc: 0.8098 - val_loss: 0.4319 - val_acc: 0.8079


In [73]:
from keras.models import load_model
best_model = load_model(data_dir+'model.m')

In [74]:
predictions = best_model.predict(x_test)
print(y_test[0])
print(predictions[0])

[1 0 0 0 1 0 0 0 0 0 1]
[ 0.35567087  0.09666565  0.34400117  0.0508016   0.462632    0.1938985
  0.3666386   0.13009974  0.27200219  0.04484326  0.08766485]


In [77]:
evaluate(predictions,y_test)


F1: 0.41910631741140214
Precision: 0.681704260651629
Recall: 0.3025583982202447
Completely correct: 1


In [ ]:
F1: 0.4036478984932593
Precision: 0.6559278350515464
Recall: 0.29152348224513175
Completely correct: 2


F1: 0.36789027833803956
Precision: 0.6961832061068702
Recall: 0.25
Completely correct: 1


Unweighted 
	F1: 0.27632915678970543
	Precision: 0.3633125556544969
	Recall: 0.22295081967213115
	Completely correct: 2

In [ ]:
Softmax
F1: 0.13215859030837004
Precision: 0.5844155844155844
Recall: 0.07450331125827815
Completely correct: 1

#Todo:Functional API class:weights 0,1 11 outputs
"It depends on your application. Class weights are useful when training on highly skewed data sets; for example, a classifier to detect fraudulent transactions. Sample weights are useful when you don't have equal confidence in the samples in your batch. A common example is performing regression on measurements with variable uncertainty." https://stackoverflow.com/questions/43459317/keras-class-weight-vs-sample-weights-in-the-fit-generator

            class_weight: Optional dictionary mapping class indices (integers)
                to a weight (float) value, used for weighting the loss function
                (during training only).
                This can be useful to tell the model to
                "pay more attention" to samples from
                an under-represented class.
            sample_weight: Optional Numpy array of weights for
                the training samples, used for weighting the loss function
                (during training only). You can either pass a flat (1D)
                Numpy array with the same length as the input samples
                (1:1 mapping between weights and samples),
                or in the case of temporal data,
                you can pass a 2D array with shape
                `(samples, sequence_length)`,
                to apply a different weight to every timestep of every sample.
                In this case you should make sure to specify
                `sample_weight_mode="temporal"` in `compile()`.


sample weights mode="temporal"
matrix mitgewicht für jede klasse in jedem sample